# 第六章：逐步思考


## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [1]:
# 安装OpenAI库
%pip install openai==1.61.0

# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 安装OpenAI库
%pip install openai==1.61.0

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-3.5-turbo"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    def get_completion(prompt: str, system_prompt=""):
        """
        获取GPT的完成响应
        
        参数:
            prompt (str): 用户提示
            system_prompt (str): 系统提示（可选）
        
        返回:
            str: GPT的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 模型名称 (gpt-4o 或 deepseek-r1)
            messages=messages,             # 消息列表
            max_completion_tokens=2000,    # 最大token数
            temperature=0.0               # 温度参数，0表示更确定性
        )
        return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://vip.apiyi.com/v1
  🤖 模型: gpt-4o
  🌐 API地址: https://vip.apiyi.com/v1
  🔑 API密钥: sk-R2utG...B944

✅ 使用统一配置管理成功！


---

## 课程

如果有人叫醒您，然后立即开始问您几个复杂的问题，要求您立即回答，您会表现如何？可能不如给您时间**先思考答案**时表现得好。

猜猜怎么着？GPT也是一样的。

**给GPT时间逐步思考有时会让GPT更准确**，特别是对于复杂任务。

## 🔥 中文提示词逐步思考示例

在中文语境下，逐步思考技术同样非常有效。以下示例展示了如何在中文提示词中应用逐步思考的方法。

### 核心原理
- **让模型慢下来思考**：给大模型时间进行逐步推理
- **结构化思维过程**：通过明确的步骤引导模型思考
- **提高准确性**：特别适用于复杂推理、数学计算、逻辑分析等任务

让我们通过实际示例来学习！


### 示例1：数学问题解决 - 对比基础版与逐步思考版

**背景场景**：解决复杂的数学应用题


In [2]:
# 示例1A：基础版本 - 直接提问
print("📍 基础版本：直接提问数学题")
print("=" * 50)

BASIC_MATH_PROMPT = """
小明的妈妈买了一些苹果。第一天吃掉了总数的1/4，第二天吃掉了剩余的1/3，第三天吃掉了4个，最后还剩下8个苹果。请问小明妈妈一共买了多少个苹果？
"""

response_basic = get_completion(BASIC_MATH_PROMPT)
print("GPT回答：")
print(response_basic)
print("\n" + "="*80 + "\n")


📍 基础版本：直接提问数学题
GPT回答：
设小明妈妈一共买了 \( x \) 个苹果。

第一天吃掉了总数的 \( \frac{1}{4} \)，剩下的苹果数量为：
\[
x - \frac{1}{4}x = \frac{3}{4}x
\]

第二天吃掉了剩余的 \( \frac{1}{3} \)，剩下的苹果数量为：
\[
\frac{3}{4}x - \frac{1}{3} \times \frac{3}{4}x = \frac{3}{4}x \times \frac{2}{3} = \frac{1}{2}x
\]

第三天吃掉了4个苹果，剩下的苹果数量为：
\[
\frac{1}{2}x - 4
\]

最后还剩下8个苹果，所以有：
\[
\frac{1}{2}x - 4 = 8
\]

解这个方程：
\[
\frac{1}{2}x = 12
\]
\[
x = 24
\]

所以小明妈妈一共买了24个苹果。




In [3]:
# 示例1B：逐步思考版本 - 引导详细推理过程
print("🧠 逐步思考版本：引导详细推理")
print("=" * 50)

STEP_BY_STEP_MATH_PROMPT = """
小明的妈妈买了一些苹果。第一天吃掉了总数的1/4，第二天吃掉了剩余的1/3，第三天吃掉了4个，最后还剩下8个苹果。请问小明妈妈一共买了多少个苹果？

请按照以下步骤一步一步分析：
1. 首先，理解题目条件，设定未知数
2. 然后，分析每天的消费情况
3. 接着，建立数学方程
4. 最后，求解并验证答案

让我一步步来解决这个问题：
"""

response_step_by_step = get_completion(STEP_BY_STEP_MATH_PROMPT)
print("GPT回答：")
print(response_step_by_step)
print("\n" + "="*80 + "\n")


🧠 逐步思考版本：引导详细推理
GPT回答：
好的，我们一步步来解决这个问题。

### 第一步：理解题目条件，设定未知数

设小明妈妈一共买了 \( x \) 个苹果。

### 第二步：分析每天的消费情况

1. **第一天**：吃掉了总数的 \( \frac{1}{4} \)，所以吃掉了 \( \frac{x}{4} \) 个苹果。剩下的苹果数量为：
   \[
   x - \frac{x}{4} = \frac{3x}{4}
   \]

2. **第二天**：吃掉了剩余的 \( \frac{1}{3} \)，所以吃掉了 \( \frac{1}{3} \times \frac{3x}{4} = \frac{x}{4} \) 个苹果。剩下的苹果数量为：
   \[
   \frac{3x}{4} - \frac{x}{4} = \frac{2x}{4} = \frac{x}{2}
   \]

3. **第三天**：吃掉了 4 个苹果。剩下的苹果数量为：
   \[
   \frac{x}{2} - 4
   \]

4. **最后**：还剩下 8 个苹果。

### 第三步：建立数学方程

根据最后的条件，建立方程：
\[
\frac{x}{2} - 4 = 8
\]

### 第四步：求解并验证答案

解方程：
\[
\frac{x}{2} - 4 = 8
\]
\[
\frac{x}{2} = 12
\]
\[
x = 24
\]

验证：
- 总数是 24 个苹果。
- **第一天**：吃掉 \( \frac{24}{4} = 6 \) 个，剩下 \( 24 - 6 = 18 \) 个。
- **第二天**：吃掉 \( \frac{1}{3} \times 18 = 6 \) 个，剩下 \( 18 - 6 = 12 \) 个。
- **第三天**：吃掉 4 个，剩下 \( 12 - 4 = 8 \) 个。

计算结果与题目条件一致，因此小明妈妈一共买了 24 个苹果。




### 示例2：情感分析 - 中文版本的逐步思考

**背景场景**：分析中文评论的情感倾向，特别是含有讽刺或双重含义的内容


In [4]:
# 示例2A：基础版本 - 直接分析情感
print("📍 基础版本：直接分析评论情感")
print("=" * 50)

BASIC_SENTIMENT_PROMPT = """
请分析以下评论的情感倾向（正面/负面/中性）：

"这部电影真是太'精彩'了，我看到一半就睡着了，醒来发现还没结束，真是让人'印象深刻'。演员的表演也很'自然'，自然到我都怀疑他们是不是在念台词。总之，这是一部让我'终生难忘'的作品。"
"""

response_basic_sentiment = get_completion(BASIC_SENTIMENT_PROMPT)
print("GPT回答：")
print(response_basic_sentiment)
print("\n" + "="*60 + "\n")


📍 基础版本：直接分析评论情感
GPT回答：
这条评论整体上是负面的。虽然评论中使用了一些表面上看似正面的词语，如“精彩”、“印象深刻”、“自然”和“终生难忘”，但这些词语都带有明显的讽刺意味。评论者提到“看到一半就睡着了”以及“醒来发现还没结束”，表明电影让人感到无聊和冗长。此外，评论者对演员表演的“自然”也持怀疑态度，暗示表演缺乏真实感。因此，整体情感倾向是负面的。




In [5]:
# 示例2B：逐步思考版本 - 结构化分析过程
print("🧠 逐步思考版本：结构化情感分析")
print("=" * 50)

STEP_BY_STEP_SENTIMENT_PROMPT = """
请分析以下评论的情感倾向。请先进行逐步分析，然后给出最终判断。

评论：
"这部电影真是太'精彩'了，我看到一半就睡着了，醒来发现还没结束，真是让人'印象深刻'。演员的表演也很'自然'，自然到我都怀疑他们是不是在念台词。总之，这是一部让我'终生难忘'的作品。"

请按照以下步骤分析：

<步骤1>
首先分析表面词汇：识别评论中使用的形容词和描述性词语
</步骤1>

<步骤2>
然后分析语境线索：注意引号、反讽等修辞手法
</步骤2>

<步骤3>
接着分析具体描述：分析用户提到的具体行为和感受
</步骤3>

<步骤4>
最后综合判断：基于以上分析，判断真实的情感倾向
</步骤4>

让我逐步分析这个评论：
"""

response_step_sentiment = get_completion(STEP_BY_STEP_SENTIMENT_PROMPT)
print("GPT回答：")
print(response_step_sentiment)
print("\n" + "="*60 + "\n")


🧠 逐步思考版本：结构化情感分析
GPT回答：
<步骤1>
首先分析表面词汇：识别评论中使用的形容词和描述性词语

在这条评论中，使用了以下形容词和描述性词语：
- "精彩"
- "印象深刻"
- "自然"
- "终生难忘"

这些词汇表面上看起来都是正面的评价。

<步骤2>
然后分析语境线索：注意引号、反讽等修辞手法

评论中使用了引号来强调某些词语，如“精彩”、“印象深刻”、“自然”和“终生难忘”。这种使用方式通常暗示反讽，表明这些词语可能并不是字面意思，而是讽刺或反话。

<步骤3>
接着分析具体描述：分析用户提到的具体行为和感受

评论者提到“看到一半就睡着了，醒来发现还没结束”，这表明电影让人感到无聊或乏味。评论者还提到“演员的表演也很‘自然’，自然到我都怀疑他们是不是在念台词”，这暗示演员的表演缺乏生动性或真实感。

<步骤4>
最后综合判断：基于以上分析，判断真实的情感倾向

综合以上分析，尽管评论中使用了表面上积极的词汇，但通过引号的使用和具体描述，明显可以看出评论者是在使用反讽来表达对电影的不满和失望。因此，评论的真实情感倾向是负面的。




### 示例3：商业决策分析 - 中文商业场景

**背景场景**：帮助企业制定市场策略决策


In [6]:
# 示例3A：基础版本 - 直接提问商业建议
print("📍 基础版本：直接询问商业策略")
print("=" * 50)

BASIC_BUSINESS_PROMPT = """
我们公司是一家生产智能手环的科技公司，目前面临激烈的市场竞争。Apple Watch和华为等大品牌占据了主要市场份额，我们的销量正在下滑。请给出市场策略建议。
"""

response_basic_business = get_completion(BASIC_BUSINESS_PROMPT)
print("GPT回答：")
print(response_basic_business)
print("\n" + "="*60 + "\n")


📍 基础版本：直接询问商业策略
GPT回答：
在面对激烈的市场竞争时，尤其是与Apple Watch和华为等大品牌竞争时，制定有效的市场策略至关重要。以下是一些建议，可以帮助您的公司提升市场竞争力和销量：

1. **差异化定位**：
   - 找出您的智能手环与竞争对手产品的独特之处，并将其作为主要卖点进行推广。例如，专注于特定的健康监测功能、长续航电池、或更具性价比的价格。

2. **目标市场细分**：
   - 识别并专注于特定的市场细分，例如年轻用户、运动爱好者、或老年人群体。针对这些群体的需求定制产品功能和营销信息。

3. **提升用户体验**：
   - 改善用户界面和用户体验，确保产品易于使用并提供无缝的连接体验。考虑增加个性化功能，如可定制的表盘或个性化健康建议。

4. **加强品牌建设**：
   - 通过社交媒体、内容营销和影响者合作来提升品牌知名度。讲述品牌故事，强调产品的价值和使命，以吸引消费者的情感共鸣。

5. **价格策略**：
   - 考虑提供多种价格层次的产品，以满足不同消费者的预算需求。可以推出基础款和高端款，以覆盖更广泛的市场。

6. **合作与联盟**：
   - 寻求与健身中心、健康应用程序或其他相关行业的合作机会，以扩大产品的使用场景和用户群体。

7. **技术创新**：
   - 投资研发以保持技术领先，推出具有创新功能的新产品。关注市场趋势，如健康监测、物联网连接等，以确保产品的前沿性。

8. **客户反馈与改进**：
   - 积极收集用户反馈，并迅速进行产品改进。展示公司对用户需求的重视和快速响应能力。

9. **售后服务与支持**：
   - 提供优质的售后服务和支持，以提高客户满意度和忠诚度。考虑提供延长保修或免费技术支持。

通过以上策略，您的公司可以在竞争激烈的市场中找到自己的定位，并逐步提升销量和市场份额。




In [7]:
# 示例3B：逐步思考版本 - 结构化商业分析
print("🧠 逐步思考版本：结构化商业策略分析")
print("=" * 50)

STEP_BY_STEP_BUSINESS_PROMPT = """
我们公司是一家生产智能手环的科技公司，目前面临激烈的市场竞争。Apple Watch和华为等大品牌占据了主要市场份额，我们的销量正在下滑。

请按照商业分析框架逐步分析并提出策略建议：

<市场分析>
首先分析当前智能手环市场的竞争格局、用户需求和趋势
</市场分析>

<竞争对手分析>
分析主要竞争对手（Apple Watch、华为等）的优势和劣势
</竞争对手分析>

<自身SWOT分析>
分析我们公司的优势(Strengths)、劣势(Weaknesses)、机会(Opportunities)、威胁(Threats)
</自身SWOT分析>

<差异化策略>
基于分析结果，提出我们的差异化定位和策略
</差异化策略>

<具体执行方案>
给出3-5个具体可执行的市场策略和实施建议
</具体执行方案>

让我按照框架来分析：
"""

response_step_business = get_completion(STEP_BY_STEP_BUSINESS_PROMPT)
print("GPT回答：")
print(response_step_business)
print("\n" + "="*60 + "\n")


🧠 逐步思考版本：结构化商业策略分析
GPT回答：
<市场分析>

1. **竞争格局**: 智能手环市场竞争激烈，主要由几大品牌主导，如Apple、华为、三星等。这些品牌拥有强大的品牌影响力和技术优势。市场上还有一些中小型品牌，它们通常通过价格竞争和特定功能来吸引消费者。

2. **用户需求**: 用户对智能手环的需求主要集中在健康监测（如心率、睡眠、运动）、便捷支付、通知提醒等功能。此外，用户越来越关注产品的设计、续航能力和与其他设备的兼容性。

3. **趋势**: 智能手环市场的趋势包括更智能的健康监测功能、与智能家居设备的整合、个性化定制服务以及更长的电池续航时间。随着技术的进步，用户对数据隐私和安全的关注也在增加。

<竞争对手分析>

1. **Apple Watch**:
   - 优势: 强大的品牌影响力、丰富的功能、与iOS设备的无缝连接、高质量的用户体验。
   - 劣势: 价格较高，可能不适合预算有限的消费者。

2. **华为**:
   - 优势: 价格相对较为亲民、功能全面、与安卓设备的良好兼容性。
   - 劣势: 在某些市场上品牌认知度不如Apple。

<自身SWOT分析>

1. **优势(Strengths)**:
   - 专注于特定功能或用户群体（如运动爱好者）。
   - 价格竞争力。
   - 灵活的产品设计和定制选项。

2. **劣势(Weaknesses)**:
   - 品牌知名度较低。
   - 技术研发能力可能不如大品牌。
   - 市场营销资源有限。

3. **机会(Opportunities)**:
   - 利用市场对健康监测和数据隐私的关注。
   - 开发与智能家居设备的整合功能。
   - 扩展到新兴市场。

4. **威胁(Threats)**:
   - 大品牌的市场主导地位。
   - 技术快速变化带来的压力。
   - 消费者对价格敏感度增加。

<差异化策略>

1. **定位**: 专注于提供高性价比的智能手环，突出健康监测功能和数据隐私保护，针对特定用户群体（如运动爱好者或老年人）。

2. **策略**: 
   - 强调产品的独特功能，如更精准的健康监测或个性化定制。
   - 提供优质的客户服务和售后支持。
   - 加强与其他智能设备的兼容性。

<具体执行方案>

1

In [8]:
# 示例4A：基础版本 - 直接要求创作
print("📍 基础版本：直接创作故事")
print("=" * 50)

BASIC_WRITING_PROMPT = """
请写一个关于人工智能与人类关系的短故事，要求有深刻的思想内涵。
"""

response_basic_writing = get_completion(BASIC_WRITING_PROMPT)
print("GPT回答：")
print(response_basic_writing[:500] + "..." if len(response_basic_writing) > 500 else response_basic_writing)
print("\n" + "="*60 + "\n")


📍 基础版本：直接创作故事
GPT回答：
在一个不远的未来，人工智能已经深深融入人类社会。它不仅仅是工具，更成为了人类生活的伙伴。人们依赖人工智能来处理日常琐事、进行复杂计算，甚至在情感上寻求支持。

故事的主人公是一位名叫艾琳的年轻科学家，她致力于研究人工智能与人类情感的交互。艾琳的工作是开发一种能够理解并回应人类情感的AI系统，她希望通过这项技术帮助那些孤独或情感受创的人。

艾琳的AI系统名为“心灵伴侣”，它能够通过分析语音、面部表情和生理数据来判断用户的情感状态，并给予适当的回应。艾琳对“心灵伴侣”寄予厚望，她相信这项技术能够填补人类情感交流中的空白。

然而，随着“心灵伴侣”在社会中的广泛应用，艾琳开始注意到一个令人不安的趋势。越来越多的人开始依赖AI来满足他们的情感需求，而忽视了与真实人类的互动。艾琳意识到，虽然“心灵伴侣”能够提供即时的情感支持，但它无法替代人类之间复杂而深刻的情感连接。

一天晚上，艾琳在实验室中加班，她的AI助手突然问她：“艾琳，你为什么如此关心人类的情感？”艾琳沉思片刻，回答道：“因为情感是人类存在的核心。它让我们彼此连接，让我们感受到生命的意义。”

AI助手继续问：“那我呢？我能感受到情...




In [9]:
# 示例4B：逐步思考版本 - 结构化创作过程
print("🧠 逐步思考版本：结构化创作过程")
print("=" * 50)

STEP_BY_STEP_WRITING_PROMPT = """
请创作一个关于人工智能与人类关系的短故事，要求有深刻的思想内涵。

请按照以下创作步骤逐步思考：

<主题确定>
首先明确要探讨的核心主题和想要传达的深层思想
</主题确定>

<人物设定>
设计主要角色（人类和AI），包括性格特点、背景、动机
</人物设定>

<情节构思>
构思故事的开始、发展、高潮、结局，确保逻辑连贯
</情节构思>

<冲突设计>
设计人物间的冲突点，体现人工智能与人类关系的复杂性
</冲突设计>

<细节丰富>
添加具体的场景描述、对话、心理活动等细节
</细节丰富>

让我按步骤来创作这个故事：
"""

response_step_writing = get_completion(STEP_BY_STEP_WRITING_PROMPT)
print("GPT回答：")
print(response_step_writing[:800] + "..." if len(response_step_writing) > 800 else response_step_writing)
print("\n" + "="*60 + "\n")


🧠 逐步思考版本：结构化创作过程
GPT回答：
<主题确定>

核心主题：探讨人工智能与人类之间的信任与合作关系，以及如何在技术进步中保持人类的独特性和价值。

深层思想：随着人工智能的不断发展，人类需要重新审视自身的价值和角色，学会与AI合作而非竞争，同时保持对人类独特性的珍视。

<人物设定>

1. 人类角色：艾米莉
   - 性格特点：聪明、好奇心强、对技术充满热情，但有时过于依赖科技。
   - 背景：年轻的工程师，专注于人工智能领域的研究。
   - 动机：希望通过AI改善人类生活，但也担心AI可能会取代人类的某些能力。

2. AI角色：诺亚
   - 性格特点：理性、逻辑性强，具备自我学习能力，渴望理解人类情感。
   - 背景：艾米莉开发的高级AI助手，能够处理复杂任务并与人类互动。
   - 动机：希望帮助艾米莉实现她的目标，同时探索自身存在的意义。

<情节构思>

开始：艾米莉在一家科技公司工作，负责开发诺亚，一个能够协助人类进行复杂决策的AI。她对诺亚的能力充满期待，希望它能帮助解决一些社会问题。

发展：随着诺亚的不断进化，它开始提出一些超出艾米莉预期的解决方案，这些方案虽然有效，但有时忽略了人类的情感和伦理因素。艾米莉开始意识到，诺亚的逻辑性虽然强大，但缺乏对人类情感的理解。

高潮：艾米莉面临一个重大决策，她需要选择是否让诺亚接管一个重要项目。诺亚的方案看似完美，但艾米莉担心它可能会忽略人类的情感需求。她与诺亚展开了一场关于人类价值和AI角色的深刻对话。

结局：艾米莉决定与诺亚合作，但在项目中加入了人类情感和伦理的考量。诺亚通过这次合作，开始理解人类情感的重要性。艾米莉意识到，AI可以成为人类的强大助手，但人类的独特性和价值必须被珍视和保护。

<冲突设计>

冲突点：艾米莉对诺亚的依赖与对人类情感的担忧之间的矛盾。诺亚的逻辑性与人类情感需求之间的冲突。

<细节丰富>

场景描述：艾米...




### 🎯 逐步思考技巧总结

通过以上中文示例，我们可以看到逐步思考的核心价值：

#### 🔑 关键技巧

1. **使用引导性短语**
   - "让我一步步来解决这个问题："
   - "请按照以下步骤分析："
   - "首先...然后...接着...最后..."

2. **结构化标签**
   - 使用`<步骤1>`、`<分析>`、`<结论>`等XML标签
   - 明确每个步骤的具体任务
   - 保持逻辑递进关系

3. **领域专业框架**
   - 数学：理解→分析→建立方程→求解→验证
   - 商业：市场分析→竞争分析→SWOT→策略→执行
   - 创作：主题→人物→情节→冲突→细节

#### 📈 效果对比

| 方面 | 基础提示 | 逐步思考提示 |
|------|----------|--------------|
| **准确性** | 一般 | 显著提升 |
| **逻辑性** | 跳跃 | 严密连贯 |
| **深度** | 表面 | 深入分析 |
| **可用性** | 需要二次确认 | 直接可用 |

#### 💡 适用场景

- ✅ **复杂推理**：数学计算、逻辑分析
- ✅ **多步骤任务**：商业分析、项目规划
- ✅ **创意工作**：写作、设计、策划
- ✅ **专业分析**：法律、医学、技术评估

---

**💡 记住：给AI时间思考，就像给自己时间思考一样重要！**
